In [4]:
import ezdxf

doc = ezdxf.readfile("../data/centerline.dxf")
msp = doc.modelspace()

In [5]:
count = {}
for e in msp:
    count[type(e)] = count.get(type(e),0)+1
print(count)

# polyline is the outside

{<class 'ezdxf.entities.lwpolyline.LWPolyline'>: 1}


In [3]:
from trajectory_generation.utils import lwpolyline2ring, lwpolyline2string
import shapely.geometry as geom
import ezdxf

class Floorplan:
    def __init__(self, floorplan_fpath, centerline_fpath):
        floor = self._get_polyline_from_dxf(floorplan_fpath)
        self.floor = geom.Polygon(shell=lwpolyline2ring(floor))

        centerline = self._get_polyline_from_dxf(centerline_fpath)
        self.centerline = lwpolyline2string(centerline)

    def _get_polyline_from_dxf(self, fpath):
        """
        load a single polyline from dxf, assumes there is at least one
        """
        doc = ezdxf.readfile(fpath)
        msp = list(doc.modelspace())
        entity = msp[0]
        return entity

    def is_inside(self, x, y):
        """given x, y coords, check if they are inside the track (between inner and outer)"""
        point = geom.Point(x, y)
        return self.floor.contains(point)

    def distance_to_centerline(self, x, y):
        point = geom.Point(x, y)
        return self.centerline.distance(point)


floorplan = Floorplan(floorplan_fpath="../data/floorplan.dxf", centerline_fpath="../data/centerline.dxf")